In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("new_iso1_group_names.csv")  

In [3]:
df_regroup = df.copy()

In [ ]:
# Step to Identify reference Quartz-based blends (PC/PLC + Quartz only)
reference_blends = df_regroup[
    (df_regroup['Blend Type'] == 'Binary') &
    (df_regroup['QUARTZ'] > 0)
]


In [5]:
# Step to Create a reference heat lookup dictionary
reference_lookup = {}
for _, row in reference_blends.iterrows():
    binder_type = 'PC' if row['PC'] > 0 else 'PLC'
    binder_pct = int(row[binder_type])
    reference_lookup[(binder_type, binder_pct)] = row['Cumulative Heat (J/g)']

In [6]:
def assign_reference(row):
    if row['Blend Type'] != 'Ternary':
        return None
    binder_type = 'PC' if row['PC'] > 0 else 'PLC'
    binder_pct = int(row[binder_type])
    return reference_lookup.get((binder_type, binder_pct), None)

In [7]:
df_regroup['Reference_Heat'] = df_regroup.apply(assign_reference, axis=1)

In [8]:
df_regroup['Delta_Heat'] = df_regroup['Cumulative Heat (J/g)'] - df_regroup['Reference_Heat']

In [9]:
# Step 7: Generate SCM ratio group labels (e.g., PC-30 | GPP=3:SC=4)
def scm_ratio_label(row):
    if row['Blend Type'] != 'Ternary':
        return None
    binder_type = 'PC' if row['PC'] > 0 else 'PLC'
    binder_pct = int(row[binder_type])

    scms = ['GPP', 'SC', 'CC', 'FA', 'VA', 'GBA', 'FBC']
    used_scms = {k: row[k] for k in scms if row[k] > 0}
    total = sum(used_scms.values())
    if total == 0:
        return f"{binder_type}-{binder_pct} | No SCMs"

    norm_ratios = {k: int(round(v / total * 10)) for k, v in used_scms.items()}
    ratio_str = ":".join([f"{k}={v}" for k, v in norm_ratios.items()])
    return f"{binder_type}-{binder_pct} | {ratio_str}"

In [10]:
df_regroup['Group_Label'] = df_regroup.apply(scm_ratio_label, axis=1)

In [15]:
ternary_summary = df_regroup[df_regroup['Blend Type'] == 'Ternary'][[
    'Original Blend Name', 'Group_Label', 'Cumulative Heat (J/g)', 'Delta_Heat'
]].sort_values(by='Group_Label')

In [16]:
print(ternary_summary.head(10)) 

           Original Blend Name          Group_Label  Cumulative Heat (J/g)  \
8    10% PC + 40% GPP + 50% SC   PC-10 | GPP=4:SC=6                 115.70   
9    20% PC + 40% GPP + 40% SC   PC-20 | GPP=5:SC=5                 149.98   
40  30% PC + 10% GPP + 60% FBC  PC-30 | GPP=1:FBC=9                 173.40   
34  30% PC + 10% GPP + 60% GBA  PC-30 | GPP=1:GBA=9                 164.78   
17   30% PC + 10% GPP + 60% SC   PC-30 | GPP=1:SC=9                 222.55   
28   30% PC + 10% GPP + 60% VA   PC-30 | GPP=1:VA=9                 154.31   
41  30% PC + 20% GPP + 50% FBC  PC-30 | GPP=3:FBC=7                 170.58   
35  30% PC + 20% GPP + 50% GBA  PC-30 | GPP=3:GBA=7                 162.18   
16   30% PC + 20% GPP + 50% SC   PC-30 | GPP=3:SC=7                 194.64   
29   30% PC + 20% GPP + 50% VA   PC-30 | GPP=3:VA=7                 154.12   

    Delta_Heat  
8          NaN  
9          NaN  
40       28.01  
34       19.39  
17       77.16  
28        8.92  
41       25.19  
35   